In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import ipywidgets as widgets
image = cv2.imread("images/board.jpg", 0)

def imshow(title="", image= None):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_image)
    plt.title(title)
    plt.axis('off')
    plt.show()

@widgets.interact(s = (0, 200, 1), r = (0, 1, 0.1))
def enhance(s,r):
    enhanced = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,1)
    plt.imshow(enhanced)
    plt.title("original image")

    detailImg = cv2.detailEnhance(enhanced, sigma_s = s, sigma_r = r)
    plt.subplot(1,2,2)
    plt.imshow(detailImg)
    plt.title("enhanced image")

    # plt.show()

In [ ]:
image = cv2.imread("images/board.jpg", cv2.IMREAD_GRAYSCALE)
threshold_value = 127
# imshow("original Image", image)
_, binary_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY_INV)
@widgets.interact(x=(0,15),y=(0,15))
def blur(x, y):
    blur = cv2.GaussianBlur(image, (x,y), 0)
    _, th3 = cv2.threshold(blur, threshold_value, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    imshow("Guassian Otsu's Thresholding", th3)
imshow("Binary Image", binary_image)

In [ ]:
# dst = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)
# imshow("Thresholded Image", dst)
image = cv2.imread("images/font.png")
imshow("original", image)
kernel = np.ones((5,5), np.uint8)
eroded = cv2.erode(image, kernel, iterations=1)
dilation = cv2.dilate(image, kernel, iterations=1)
OPENED = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel, iterations=1)
CLOSED = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=1)
imshow("Erroded image", eroded)
imshow("dilation", dilation)
# imshow("opened", OPENED)
# imshow("closed", CLOSED)

In [ ]:
image = cv2.imread("images/finger.jpg")
@widgets.interact(thresh1 = (0, 255))
def interact(thresh1):
    edge = cv2.Canny(image, 100, thresh1)
    imshow("detected", edge)

In [ ]:
image = cv2.imread("images/board.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, th2 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# imshow('After thresholding', th2)
contours, hierarchy = cv2.findContours(th2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(image, contours, -1, (0,255,0), thickness = 15)
# imshow('Contours overlaid on original image', image)

# print("Number of Contours found = " + str(len(contours)))

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

contour, higherarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img_copy = image.copy()
img  = cv2.drawContours(img_copy, contour, -1, (0,255,0), thickness = 5)
imshow("contour_image", img)


In [ ]:
image = cv2.imread("images/board.jpg")
orig_image = image.copy()
rect_image = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contour, hiegherarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
img_copy = image.copy()
draw_img = cv2.drawContours(img_copy, contour, -1, (0, 255, 0), 3)
print("Numbers of contour", len(contour))


def getContourArea(contour):
    all_area = []
    for cnt in contour:
        area = cv2.contourArea(cnt)
        all_area.append(area)
    return all_area


print("contour area before sorting")
print(getContourArea(contour))
sorted_area = sorted(contour, key=cv2.contourArea, reverse=False)
print("contour area After sorting")
print(getContourArea(sorted_area))

for (i, c) in enumerate(sorted_area):
    M = cv2.moments(c)
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    cv2.putText(image, str(i+1), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
    cv2.circle(image,(cx,cy), 5, (0,0,255), -1)
    cv2.drawContours(image, [c], -1, (0,0,255), 3)

imshow('Contours by area', image)

In [ ]:
for c in contour:
    x,y,w,h= cv2.boundingRect(c)
    cv2.rectangle(rect_image, (x,y), (x+w, y+h), (0,0,255),2)
    cv2.drawContours(orig_image, c, -1, (0, 255, 0), 3)

for c in contours:
    accuracy = 0.03 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, accuracy, True)
    cv2.drawContours(img_copy, [approx], 0, (0, 255, 0), 2)

imshow('original image', orig_image)
imshow('bounded image', rect_image)
imshow('approxy Poly DP image', img_copy)

In [ ]:
image = cv2.imread("images/board.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contour, higherarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
copy = image.copy()
orginal_image =image.copy()
cv2.drawContours(copy, contour, -1, (0,255,0), 3)
imshow("contour hand", copy)

n = len(contour) - 1
contours = sorted(contour, key=cv2.contourArea, reverse=False)[:n]
for c in contour:
    hull = cv2.convexHull(c)
    cv2.drawContours(orginal_image, [hull], 0, (0, 255, 0), 2)
    
imshow("convex hull", orginal_image)

In [ ]:
template = cv2.imread("images/star.png")
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(template_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contour, higherarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
sorted_contour = sorted(contour, key=cv2.contourArea, reverse=True)

template_contour = sorted_contour[1]

target = cv2.imread("images/polygons.png")
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(target_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contour, higherarchy = cv2.findContours(thresh,cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

n = len(contour) - 1
sorted_contour = sorted(contour, key=cv2.contourArea, reverse=False)[:n]
for c in sorted_contour:
    match = cv2.matchShapes(template_contour, c, cv2.CONTOURS_MATCH_I3, 0)
    print(match)
    if match < 0.15:
        match_contour = c
        break
cv2.drawContours(target, [match_contour], 0, (0,255,0), 3)
imshow("output", target)

In [ ]:
image = cv2.imread('images/houghline.png')
# imshow('Original', image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 100, 170, apertureSize = 3)
lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)
if lines is not None:
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

imshow('Hough Lines', image)

In [ ]:
image = cv2.imread('images/chesse.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 100, 170, apertureSize = 3)
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 10, 3, 25)
print(lines.shape)

for x in range(0, len(lines)):
    for x1,y1,x2,y2 in lines[x]:
        cv2.line(image,(x1,y1),(x2,y2),(0,255,0),2)

imshow('Probabilistic Hough Lines', image)

In [ ]:
image = cv2.imread('images/circle.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 25)


if circles is not None:
    circles = np.uint16(np.around(circles))
    print(circles)
    for i in circles[0]:
        center = (i[0], i[1])
        radius = i[2]
        cv2.circle(image,center, radius, (0, 0, 255), 5)
        cv2.circle(image,center, 2, (0, 0, 255), 5)
else:
    print("center is not detected!!")

imshow('otput', image)


In [ ]:
image = cv2.imread('images/sunflowers.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

detect = cv2.SimpleBlobDetector_create().detect(image)
blank = np.zeros((1,1), np.uint8)
blobs = cv2.drawKeypoints(image, detect, blank, (0,255,0), cv2.DRAW_MATCHES_FLAGS_DEFAULT)

imshow("Blobs", blobs)


In [ ]:
image = cv2.imread("images/polygon.jpg")
# imshow("original", image)
orig_image = image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
contour, hiegherarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

n = len(contour) - 1
sorted_contour = sorted(contour, key=cv2.contourArea, reverse=False)[:n]
for (i, cnt) in enumerate(sorted_contour):
    M = cv2.moments(cnt)
    cx = int(M['m10']/ M['m00'])
    cy = int(M['m01']/ M['m00'])
    # cv2.putText(orig_image, str(i+1), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1)
    cv2.circle(orig_image, (cx, cy), 3, (0, 0, 255, 1))
    cv2.drawContours(orig_image, cnt, -1, (0, 255, 0), 3)
imshow("contour", orig_image)




In [ ]:
image = cv2.imread('images/blob.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
detect = cv2.SimpleBlobDetector_create().detect(image)
copy = image.copy()

blank = np.zeros((1,1), np.uint8)
blobs = cv2.drawKeypoints(image, detect, blank, (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
total = str(len(detect))
text = "total number of blob" + total
cv2.putText(blobs, text, (20, 560), cv2.FONT_HERSHEY_SIMPLEX, 1, (100, 0, 255), 2)
imshow("Blobs", blobs)


params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.minArea = 100

params.filterByConvexity = False
params.minConvexity = 0.2

params.filterByCircularity = True
params.minCircularity = 0.9

params.filterByInertia = True
params.minInertiaRatio = 0.01


detector = cv2.SimpleBlobDetector_create(params)
keyPoint = detector.detect(gray)
blank = np.zeros((1,1), np.uint8)
blobs = cv2.drawKeypoints(image, keyPoint, blank, (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
imshow("custome blob", blobs)

In [ ]:
image = cv2.imread('images/temp.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

template = cv2.imread('images/new.png',0)
imshow('find match', template)

result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

top_left = max_loc
bottom_right = (top_left[0] + 280, top_left[1] + 100)
cv2.rectangle(image, top_left, bottom_right, (255,0,0), 5)
imshow("match shape", image)